In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename

In [19]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]

train_paths = glob('../feature/raw_main/*_train.gz')
test_paths = glob('../feature/raw_main/*_test.gz')

train_paths = [path for path in train_paths 
               if path.count('DT') 
               or path.count('Fraud') 
               or path.count('D1')
               or path.count('D3')
               or path.count('C1')
               or path.count('V221')
               or path.count('V285')
               or path.count('ID')
              ]
test_paths = [path for path in test_paths 
               if path.count('DT') 
               or path.count('Fraud') 
               or path.count('D1')
               or path.count('D3')
               or path.count('C1')
               or path.count('V221')
               or path.count('V285')
               or path.count('ID')
              ]

train_df = parallel_load_data(train_paths)
test_df = parallel_load_data(test_paths)

Process ForkPoolWorker-119:
Process ForkPoolWorker-127:
Process ForkPoolWorker-121:
Process ForkPoolWorker-118:
Process ForkPoolWorker-102:
Process ForkPoolWorker-124:
Process ForkPoolWorker-101:
Process ForkPoolWorker-129:
Process ForkPoolWorker-111:
Process ForkPoolWorker-115:
Process ForkPoolWorker-104:
Process ForkPoolWorker-106:
Process ForkPoolWorker-113:
Process ForkPoolWorker-116:
Process ForkPoolWorker-126:
Process ForkPoolWorker-122:
Process ForkPoolWorker-117:
Process ForkPoolWorker-112:
Process ForkPoolWorker-98:
Process ForkPoolWorker-103:
Process ForkPoolWorker-108:
Process ForkPoolWorker-105:
Process ForkPoolWorker-125:
Process ForkPoolWorker-107:
Process ForkPoolWorker-100:
Traceback (most recent call last):
Process ForkPoolWorker-123:
Process ForkPoolWorker-109:
Process ForkPoolWorker-114:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-128:
Process ForkPoolWorker-110:
Process ForkPoolWorker-99:
Traceback (most recent call l

In [93]:
# train_df.set_index(COLUMN_ID, inplace=True)
# test_df.set_index(COLUMN_ID, inplace=True)
data = pd.concat([train_df, test_df], axis=0)
data['C13'] += 1
list_path = glob('../output/0830_ieee__same_user__pattern-user_keys__card*')
for path in list_path:
    fname = path[56:].replace('csv', '').replace(r'.', '')
    same_user = pd.read_csv(path).set_index(COLUMN_ID)
    data[f'same_user__{fname}']  = same_user['same_user_id']

In [94]:
cols_C = ['C1', 'C13']
cols_V = ['V221', 'V285']
col_same = 'same_user__addr_pemail_M'
data.sort_values(by=[col_same, COLUMN_DT], inplace=True)

In [95]:
for no in tqdm(range(1,11,1)):

    col = 'C1'
    c_max = data.groupby(col_same)[col].max().to_frame('C1_max')
    if col_same not in data.columns:
        data.set_index(col_same, inplace=True)
    data['C1_max'] = c_max
    
    data['max_ratio'] = data['C1_max'] / data[col]
    data[f'diff_user_no_{col}'] = (data['max_ratio']>3.0)*1 + 1
    
    col = 'C13'
    if col_same not in data.columns:
        data.reset_index(inplace=True)
    data['C13_shift1'] = data.groupby(col_same)[col].shift(1)
    
    data['C13_diff_ratio'] = (data[col] / data['C13_shift1']).map(lambda x: abs(x))
    data[f'diff_user_no_{col}'] = (data['C13_diff_ratio']>3.0) * 1 + 1
    
    col_tmp_id = f'tmp_user_id__{no}'
    col_new_id = f'new_user_id__{no}'
    data[col_tmp_id] = data[[col_same, 'diff_user_no_C1', 'diff_user_no_C13']].apply(lambda x: f'{x[0]}-{x[1]}-{x[2]}', axis=1)
    new = data[col_tmp_id].drop_duplicates().sort_values().to_frame()
    new['new_id'] = np.arange(len(new))
    data.set_index(col_tmp_id, inplace=True)
    new.set_index(col_tmp_id, inplace=True)
    data[col_new_id] = new['new_id']
    data.reset_index(inplace=True)
    
    col_same = col_new_id

100%|██████████| 10/10 [06:48<00:00, 41.44s/it]


In [105]:
cols_new = [col for col in data.columns if col.count('new_user')]
data[['same_user__addr_pemail_M'] + cols_new].to_csv('../output/0831_ieee__some_new_user_id.csv')

In [108]:
data['same_user__addr_pemail_M'].unique().shape

(859532,)